In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import random
import os

os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
import tensorflow as tf

seed = 2023
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)
tf.experimental.numpy.random.seed(seed)
os.environ["TF_CUDNN_DETERMINISTIC"] = "1"
os.environ["TF_DETERMINISTIC_OPS"] = "1"
# Set a fixed value for the hash seed
os.environ["PYTHONHASHSEED"] = str(seed)
import dolfin as df
import time
from utils import *
from utils_training import *
from matplotlib.lines import Line2D

import seaborn as sns

sns.set_theme()
sns.set_context("paper")
sns.set(rc={"xtick.bottom": True, "ytick.left": True})

In [ ]:
params = np.load("./params.npy")
# (nb_layers, width, modes, level)
tmp = np.load(f"./misfits_val/misfits_0/model_0.npy")

epochs = [10 * (i + 1) for i in range(len(tmp))]
print(f"{len(params) = }   {epochs = }")

In [ ]:
nb_layers, width, modes, level = params.T
nb_layers = np.unique(nb_layers).astype(int)
width = np.unique(width).astype(int)
modes = np.unique(modes).astype(int)
level = np.unique(level).astype(int)
norm = [r"$L_2$", r"$H^1$", r"$H^2$"]
print(f"{nb_layers=}  {width=} {modes=} {level=}")
line_type = ["--o", "-s", ":>"]
colors = list(sns.color_palette("deep").as_hex())
print(colors)

if not (os.path.exists("./images")):
    os.makedirs("./images")


In [ ]:
small_data = True
data = DataLoader(small_data)

In [ ]:
def compute_error(X, Y_pred, Y_true):
    diff = Y_true - Y_pred
    error = tf.sqrt(
        tf.reduce_mean(
            (diff**2) * X[:, :, :, -1],
            axis=[1, 2],
        )
        / tf.reduce_mean(
            ((Y_true) ** 2) * X[:, :, :, -1],
            axis=[1, 2],
        )
    )
    return tf.reduce_mean(error)


In [ ]:
from generate_data import PhiFemSolver

F, Phi, G, params_data = create_FG_numpy(10, 64)
solver = PhiFemSolver(nb_cell=64 - 1, params=params_data)
W_phi_fem = solver.solve_several()
X_test = generate_manual_new_data_numpy(Phi, F / data.max_norm_F, G)

Y_true = W_phi_fem[:, :, :] * X_test[:, :, :, 1] + G


In [ ]:
color_indices = [0, 3, 4, 8]
legend_elements = [
    Line2D(
        [0],
        [0],
        color=colors[color_indices[i]],
        lw=4,
        label=f"$nk' = {width[i]}$",
    )
    for i in range(len(color_indices))
]

plt.figure(figsize=(6, 4))
for j in range(len(nb_layers)):
    indices = np.where(
        (params[:, 0] == nb_layers[j])
        & (params[:, 2] == 10)
        & (params[:, -1] == 2)
    )[0]
    print(indices)
    count = 0
    for i in indices:
        nb_layer, width_i, modes_i, level = params[i]
        nb_layer, width_i, modes_i, level = (
            int(nb_layer),
            int(width_i),
            int(modes_i),
            int(level),
        )
        agent = Agent(
            data,
            small_data,
            width=width_i,
            modes=modes_i,
            nb_layers=nb_layer,
            index_test_case=i,
            loss_level=level,
        )
        L2_errors = []
        for epoch in epochs:
            agent.model.load_weights(
                f"./models/models_{i}/model_{epoch}/model_weights"
            )
            W_pred = agent.model.call(X_test)
            Y_pred = W_pred[:, :, :, 0] * X_test[:, :, :, 1] + G
            L2_error = compute_error(X_test, Y_pred, Y_true)
            L2_errors.append(L2_error)
        plt.plot(
            epochs,
            L2_errors,
            line_type[j],
            color=colors[color_indices[count]],
            label=f"width={int(params[i,1])}",
        )
        count += 1
plt.yscale("log")
plt.legend(handles=legend_elements, ncol=2, fontsize=12)
plt.xlabel("Epochs", fontsize=16)
plt.ylabel(r"$L^2$ relative error", fontsize=16)
plt.tight_layout()
plt.savefig("./images/change_width.png")
plt.show()


In [ ]:
color_indices = [0, 3, 4]

legend_elements = [
    Line2D([0], [0], color=colors[color_indices[i]], lw=4, label=f"{norm[i]}")
    for i in range(len(color_indices))
]

plt.figure(figsize=(6, 4))
for j in range(len(nb_layers)):
    indices = np.where(
        (params[:, 0] == nb_layers[j])
        & (params[:, 2] == 10)
        & (params[:, 1] == 20)
    )[0]
    count = 0
    for i in indices:
        nb_layer, width_i, modes_i, level = params[i]
        nb_layer, width_i, modes_i, level = (
            int(nb_layer),
            int(width_i),
            int(modes_i),
            int(level),
        )
        agent = Agent(
            data,
            small_data,
            width=width_i,
            modes=modes_i,
            nb_layers=nb_layer,
            index_test_case=i,
            loss_level=level,
        )
        L2_errors = []
        for epoch in epochs:
            agent.model.load_weights(
                f"./models/models_{i}/model_{epoch}/model_weights"
            )
            W_pred = agent.model.call(X_test)
            Y_pred = W_pred[:, :, :, 0] * X_test[:, :, :, 1] + G
            L2_error = compute_error(X_test, Y_pred, Y_true)
            L2_errors.append(L2_error)
        plt.plot(
            epochs,
            L2_errors,
            line_type[j],
            color=colors[color_indices[count]],
            label=f"level={int(params[i,-1])}",
        )
        count += 1
plt.yscale("log")
plt.legend(handles=legend_elements, ncol=3, fontsize=12)
plt.xlabel("Epochs", fontsize=16)
plt.ylabel(r"$L^2$ relative error", fontsize=16)
plt.tight_layout()
plt.savefig("./images/change_level.png")
plt.show()


In [ ]:
color_indices = [0, 3, 4, 9]
legend_elements = [
    Line2D(
        [0],
        [0],
        color=colors[color_indices[i]],
        lw=4,
        label=f"{modes[i]} modes",
    )
    for i in range(len(color_indices))
]

plt.figure(figsize=(6, 4))
for j in range(len(nb_layers)):
    indices = np.where(
        (params[:, 0] == nb_layers[j])
        & (params[:, 1] == 20)
        & (params[:, -1] == 2)
    )[0]
    count = 0
    for i in indices:
        nb_layer, width_i, modes_i, level = params[i]
        nb_layer, width_i, modes_i, level = (
            int(nb_layer),
            int(width_i),
            int(modes_i),
            int(level),
        )
        agent = Agent(
            data,
            small_data,
            width=width_i,
            modes=modes_i,
            nb_layers=nb_layer,
            index_test_case=i,
            loss_level=level,
        )
        L2_errors = []
        for epoch in epochs:
            agent.model.load_weights(
                f"./models/models_{i}/model_{epoch}/model_weights"
            )
            W_pred = agent.model.call(X_test)
            Y_pred = W_pred[:, :, :, 0] * X_test[:, :, :, 1] + G
            L2_error = compute_error(X_test, Y_pred, Y_true)
            L2_errors.append(L2_error)

        plt.plot(
            epochs,
            L2_errors,
            line_type[j],
            color=colors[color_indices[count]],
            label=f"modes={int(params[i,2])}",
        )
        count += 1
plt.yscale("log")
plt.legend(handles=legend_elements, ncol=2, fontsize=12)
plt.xlabel("Epochs", fontsize=16)
plt.ylabel(r"$L^2$ relative errors", fontsize=16)
plt.tight_layout()
plt.savefig("./images/change_modes.png")
plt.show()
